In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import lightgbm as lgb

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
import seaborn as sns


In [26]:
features = pd.read_pickle("../train_features.pkl")
labels = pd.read_csv("../train_labels.csv",index_col="challenge_oid")
features.index.name = "challenge_oid"
features = features.join(labels)

In [56]:
import sklearn.preprocessing
label_encoder = sklearn.preprocessing.LabelEncoder()
encoded_labels = label_encoder.fit_transform(features["classALeRCE"])
train_dataset = lgb.Dataset(features.drop("classALeRCE", axis=1), 
                            label=encoded_labels,
                            #categorical_feature=[]
                           )

In [90]:
param = {
    'num_leaves': 31,
    'objective': 'multiclass',
    'num_class':14,
    'num_iterations':500,
    'max_depth' : 50,
    'learning_rate' : 1e-4
}

In [86]:
bst = lgb.train(param, train_dataset)

C:\Users\Alexis\Anaconda3\envs\ds\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [87]:
features_test = pd.read_pickle("../test_features.pkl")


In [88]:
prediction = bst.predict(features_test)

In [110]:
test_df = pd.DataFrame(prediction, columns=label_encoder.classes_)
test_df["challenge_oid"] = features_test.index
test_df["Outlier"] = 0.

In [111]:
test_df.to_csv("test_sub.csv", index="challenge_oid")